In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.nn import HANConv, DeepGraphInfomax, Linear
from torch_geometric.data import HeteroData
from torch_geometric.utils import dense_to_sparse


**1. Load and Process Data**


In [ ]:
#  1. 데이터 로드
ppi_df = pd.read_csv("../analysis/assets/module_assignment_network/ppi_mat.tsv.gz", sep="\t", index_col="gene_name")
tom_df = pd.read_csv("../analysis/assets/module_assignment_network/tom_mat.tsv.gz", sep="\t", index_col="gene_name")
go_hierarchy_df = pd.read_csv("../analysis/assets/module_assignment_network/go_hierarchy.txt.gz", sep="\t", index_col=0)
gene_to_go_df = pd.read_csv("../analysis/assets/module_assignment_network/gene_to_go.tsv.gz", sep="\t")
module_df = pd.read_csv("../analysis/assets/wgcna/full_module_scores.txt.gz", sep="\t")
map_cis_df = pd.read_csv("../analysis/assets/module_assignment_network//eQTL_sizes.tsv.gz", sep="\t")

#  2. Grey module 유전자 제거
valid_genes = module_df[module_df["module"] != "grey"]["gene_name"].unique()
eQTL_genes  = map_cis_df["phenotype_id"].unique()

#  3. PPI, TOM, GO 어노테이션과 공통된 유전자 추출
common_genes = sorted(set(ppi_df.index) & set(tom_df.index) & set(gene_to_go_df['Gene']) & set(valid_genes) & set(eQTL_genes))

#  4. module_df를 common_genes 순서로 정렬
module_df = module_df[module_df["gene_name"].isin(common_genes)].set_index("gene_name").reindex(common_genes).reset_index()
module_to_idx = {mod: i for i, mod in enumerate(sorted(module_df["module"].unique()))}  # Module 인덱스 매핑

map_cis_df = map_cis_df[map_cis_df["phenotype_id"].isin(common_genes)].set_index("phenotype_id").reindex(common_genes).reset_index()

#  5. PPI 및 TOM 데이터에서 common_genes만 선택
ppi_arr = ppi_df.loc[common_genes, common_genes]
tom_arr = tom_df.loc[common_genes, common_genes]


In [4]:
# 기존 module_features 와 concat (num_genes, num_modules + 1)
eqtl_features = map_cis_df.iloc[:,1:].to_numpy()
# eQTL effect size 벡터 추가 (num_genes, 1) shape
eqtl_tensor = torch.tensor(eqtl_features, dtype=torch.float)

In [ ]:
# (2) One-hot encoding
num_genes = len(common_genes)  #  수정: common_genes 기준으로 갯수 설정
unique_modules = sorted(module_df[module_df["module"] != "grey"]["module"].unique())
num_modules = len(unique_modules)
one_hot_matrix = np.zeros((num_genes, num_modules))
for i, mod in enumerate(module_df["module"]):
    one_hot_matrix[i, module_to_idx[mod]] = 1
module_features = torch.tensor(one_hot_matrix, dtype=torch.float)

In [6]:
gene_to_go_df  = gene_to_go_df[ gene_to_go_df['Gene'].isin(common_genes) ]
go_terms_with_genes = set(gene_to_go_df["GO_Term"])
filtered_go_hierarchy_df = go_hierarchy_df[
    (go_hierarchy_df["GO_term"].isin(go_terms_with_genes)) | 
    (go_hierarchy_df["Parent_GO_term"].isin(go_terms_with_genes))
].copy()

In [7]:
# GO 어노테이션 및 계층 구조에 등장하는 GO term들을 모두 포함
go_terms_from_annotation = set(gene_to_go_df['GO_Term'])
go_terms_from_hierarchy = set(filtered_go_hierarchy_df['GO_term']) | set(filtered_go_hierarchy_df['Parent_GO_term'])
all_go_terms = sorted(go_terms_from_annotation.union(go_terms_from_hierarchy))
num_go_nodes = len(all_go_terms)
go_term_to_idx = {term: i for i, term in enumerate(all_go_terms)}
gene_to_idx = {gene: i for i, gene in enumerate(common_genes)}

In [8]:
# [A] gene → GO edge 구축 (어노테이션 정보)
gene_go_source = []
gene_go_target = []
for _, row in gene_to_go_df.iterrows():
    gene = row['Gene']
    go_term = row['GO_Term']
    if gene in gene_to_idx and go_term in go_term_to_idx:
        gene_go_source.append(gene_to_idx[gene])
        gene_go_target.append(go_term_to_idx[go_term])
# edge_index shape: [2, num_edges]
gene_go_edge_index = torch.tensor([gene_go_source, gene_go_target], dtype=torch.long)

# [B] GO → gene 역방향 edge (반대 방향 추가)
go_gene_edge_index = torch.tensor([gene_go_target, gene_go_source], dtype=torch.long)

# [C] GO 계층 구조 edge 구축: child (GO_term) → parent (Parent_GO_term)
go_hierarchy_source = []
go_hierarchy_target = []
for _, row in filtered_go_hierarchy_df.iterrows():
    child = row['GO_term']
    parent = row['Parent_GO_term']
    if child in go_term_to_idx and parent in go_term_to_idx:
        go_hierarchy_source.append(go_term_to_idx[child])
        go_hierarchy_target.append(go_term_to_idx[parent])
go_hierarchy_edge_index = torch.tensor([go_hierarchy_source, go_hierarchy_target], dtype=torch.long)

# (선택사항) GO 계층 구조의 역방향 edge: parent → child
go_hierarchy_rev_edge_index = torch.tensor([go_hierarchy_target, go_hierarchy_source], dtype=torch.long)

**2. Apply Edge Thresholding**

In [9]:
PPI_THRESHOLD = 700   # STRING database recommends 700+ for high-confidence interactions
TOM_THRESHOLD = 0.10  # Keep only strong TOM connections

# Filter edges based on threshold
ppi_filtered = np.where(ppi_arr > PPI_THRESHOLD, ppi_arr, 0)
tom_filtered = np.where(tom_arr > TOM_THRESHOLD, tom_arr, 0)

# Keep only edges that pass both PPI and TOM thresholds
combined_mask = (ppi_filtered > 0) & (tom_filtered > 0) 
#ppi_filtered = np.where(combined_mask, ppi_filtered, 0)
#tom_filtered = np.where(combined_mask, tom_filtered, 0)

# Print summary of remaining edges
print(f"Remaining PPI edges: {np.count_nonzero(ppi_filtered)}")
print(f"Remaining TOM edges: {np.count_nonzero(tom_filtered)}")
print(f"Remaining intersected edges: {np.count_nonzero(combined_mask)}")


Remaining PPI edges: 44030
Remaining TOM edges: 143192
Remaining intersected edges: 7676


**3. Convert to PyTorch Format and Heterogenous Graph**

In [10]:
# Convert adjacency matrices to tensors
ppi_tensor = torch.tensor(ppi_filtered, dtype=torch.float)
tom_tensor = torch.tensor(tom_filtered, dtype=torch.float)

# Convert dense adjacency matrices to sparse format (edge lists)
ppi_edge_index, ppi_edge_weight = dense_to_sparse(ppi_tensor)
tom_edge_index, tom_edge_weight = dense_to_sparse(tom_tensor)

data = HeteroData()

# Assign one-hot encoded node features (replace with real features if available)
num_nodes = len(common_genes)
#data["gene"].x = torch.eye(num_nodes)  # Identity matrix as placeholder features
#data["gene"].x = torch.cat([data["gene"].x, module_features], dim=1)
gene_features = torch.cat([module_features, eqtl_tensor], dim=1)
data["gene"].x = gene_features

# Store gene names
data["gene"].names = common_genes

# [ii] go 노드: GO term 개수에 맞는 one-hot (임시) feature 사용
num_go_features = 64

data["go"].x = torch.nn.Embedding(num_go_nodes, num_go_features).weight 
data["go"].names = all_go_terms


# Assign edges to the graph
data["gene", "PPI", "gene"].edge_index = ppi_edge_index
data["gene", "PPI", "gene"].edge_attr = ppi_edge_weight.unsqueeze(-1) 

data["gene", "TOM", "gene"].edge_index = tom_edge_index
data["gene", "TOM", "gene"].edge_attr = tom_edge_weight.unsqueeze(-1)

In [11]:
data["gene", "has_GO", "go"].edge_index = gene_go_edge_index 
data["gene", "has_GO", "go"].edge_attr = torch.ones(gene_go_edge_index.size(1),1) 

#data["go", "rev_has_GO", "gene"].edge_index = go_gene_edge_index
#data["go", "rev_has_GO", "gene"].edge_attr = torch.ones(go_gene_edge_index.size(1), 1) 

# [v] GO 계층 구조 edge 추가 (GO → GO)
data["go", "is_a", "go"].edge_index = go_hierarchy_edge_index
data["go", "is_a", "go"].edge_attr = torch.ones(go_hierarchy_edge_index.size(1), 1)

#data["go", "rev_is_a", "go"].edge_index = go_hierarchy_rev_edge_index
#data["go", "rev_is_a", "go"].edge_attr = torch.ones(go_hierarchy_rev_edge_index.size(1), 1) / 4

# metadata (node type과 edge type) 자동 생성 혹은 명시적 정의
metadata = (list(data.node_types), list(data.edge_types))
print("노드 타입:", data.node_types)
print("엣지 타입:", data.edge_types)
data

노드 타입: ['gene', 'go']
엣지 타입: [('gene', 'PPI', 'gene'), ('gene', 'TOM', 'gene'), ('gene', 'has_GO', 'go'), ('go', 'is_a', 'go')]


HeteroData(
  gene={
    x=[2615, 12],
    names=[2615],
  },
  go={
    x=[23934, 64],
    names=[23934],
  },
  (gene, PPI, gene)={
    edge_index=[2, 44030],
    edge_attr=[44030, 1],
  },
  (gene, TOM, gene)={
    edge_index=[2, 143192],
    edge_attr=[143192, 1],
  },
  (gene, has_GO, go)={
    edge_index=[2, 146545],
    edge_attr=[146545, 1],
  },
  (go, is_a, go)={
    edge_index=[2, 29365],
    edge_attr=[29365, 1],
  }
)

In [12]:
print("Edge index dictionary keys:", data.edge_index_dict.keys())
for key, edge_index in data.edge_index_dict.items():
    print(f"{key}: shape {edge_index.shape}")
meta_paths = [
    [('gene', 'PPI', 'gene')],
    [('gene', 'TOM', 'gene')],
    # 유전자와 GO 간 어노테이션 edge를 통한 경로: gene → has_GO → go → rev_has_GO → gene
    #[('gene', 'has_GO', 'go'), ('go', 'rev_has_GO', 'gene')]#,
    [('gene', 'has_GO', 'go')],
    # (선택사항) GO 계층 구조를 포함한 경로: gene → has_GO → go → is_a → go → rev_has_GO → gene
    [('go', 'is_a', 'go')]
    #[('gene', 'has_GO', 'go'), ('go', 'is_a', 'go'), ('go', 'rev_has_GO', 'gene')]
]

Edge index dictionary keys: dict_keys([('gene', 'PPI', 'gene'), ('gene', 'TOM', 'gene'), ('gene', 'has_GO', 'go'), ('go', 'is_a', 'go')])
('gene', 'PPI', 'gene'): shape torch.Size([2, 44030])
('gene', 'TOM', 'gene'): shape torch.Size([2, 143192])
('gene', 'has_GO', 'go'): shape torch.Size([2, 146545])
('go', 'is_a', 'go'): shape torch.Size([2, 29365])


In [13]:
# Define a corruption function that shuffles node features.
# This function now accepts two arguments: x_dict and edge_index_dict.

def corruption(x_dict, edge_index_dict):
    corrupted_x = {key: x[torch.randperm(x.size(0))] for key, x in x_dict.items() if x.size(0) > 1}  # ✅ 수정: 노드 수가 1개일 경우 permutation 오류 방지
    return corrupted_x, edge_index_dict

# Readout function (summarizing the whole graph representation)
# Updated readout function to accept extra arguments.


def readout(x, *args, **kwargs):
    return torch.sigmoid(torch.mean(x, dim=0))

In [14]:
class HANEncoder(torch.nn.Module):
    def __init__(self,
                 gene_in_dim,       # 초기 gene 특성 차원 (ex: 128)
                 go_in_dim,         # 초기 go 특성 차원   (ex: 64)
                 hidden_channels,
                 out_channels,
                 metadata,
                 heads=8,
                 dropout=0.4):
        super().__init__()
        
        # [1] 첫 번째 HANConv
        in_channels_1 = {
            'gene': gene_in_dim,
            'go': go_in_dim
        }
        self.han_conv1 = HANConv(
            in_channels_1, 
            hidden_channels,
            metadata=metadata,
            heads=heads,
            dropout=dropout
        )
        
        # [2] 두 번째 Branch: TOM
        #  - TOM도 같은 원본 x_dict를 입력으로 쓴다면, 차원은 동일
        self.han_conv_tom = HANConv(
            in_channels_1,
            hidden_channels,
            metadata=metadata,
            heads=heads,
            dropout=dropout
        )
        
        # [3] 세 번째 HANConv (두 Branch 결과 결합 후)
        in_channels_2 = {
            'gene': hidden_channels * 2,  # out_general + out_tom => 2배
            'go': hidden_channels         # if 'go'는 한 가지만 살린다면 1배
        }
        self.han_conv2 = HANConv(
            in_channels_2,
            hidden_channels,
            metadata=metadata,
            heads=heads,
            dropout=dropout
        )
        
        # [4] 최종 선형 변환: (hidden_channels * heads) -> out_channels
        self.linear = Linear(hidden_channels, out_channels)
        
    def forward(self, x_dict, edge_index_dict):
        # 첫 번째 branch
        out_general = self.han_conv1(x_dict, edge_index_dict)
        
        # 두 번째 branch(TOM) => edge_index_dict 중 TOM만 사용
        tom_edge_index_dict = {
            key: edge
            for key, edge in edge_index_dict.items()
            if key == ('gene', 'TOM', 'gene')
        }
        out_tom = self.han_conv_tom(x_dict, tom_edge_index_dict)
        
        # 두 branch 결합(gene)
        merged_gene = torch.cat(
            [out_general['gene'], out_tom['gene']],
            dim=-1
        )  # shape = [num_gene_nodes, 1024]
        
        # go 임베딩은 out_general['go']만 사용 (또는 tom도 있으면 cat)
        merged_go = out_general['go']  # shape = [num_go_nodes, 512]
        
        # [5] 다음 레이어의 입력
        merged_x_dict = {
            'gene': merged_gene,
            'go': merged_go
        }
        out_merge = self.han_conv2(merged_x_dict, edge_index_dict)
        
        # [6] 최종 gene 임베딩에 선형 변환
        out_final = self.linear(out_merge['gene'])  # shape = [num_gene_nodes, out_channels]
        
        return out_final

# in_channels는 각 노드 타입별 입력 feature dimension (one-hot feature이므로 node 개수)
#in_channels = {'gene': num_genes + num_modules, 'go': num_go_nodes}
in_channels = {'gene': num_modules + 2, 'go': num_go_features}
hidden_dim = 64
out_dim = 64
heads = 8
dropout = 0.3

# metadata는 위에서 생성한 metadata 사용
encoder = HANEncoder(
    gene_in_dim=num_modules +2,
    go_in_dim=num_go_features,
    hidden_channels=hidden_dim,
    out_channels=out_dim,
    metadata=metadata,
    heads=heads,
    dropout=dropout
)

# DeepGraphInfomax 모델 구성
dgi_model = DeepGraphInfomax(
    hidden_channels=out_dim,
    encoder=encoder,
    summary=lambda x, *args, **kwargs: torch.sigmoid(torch.mean(x, dim=0)),  # readout 함수
    corruption=lambda x_dict, edge_index_dict: (
        {key: x[torch.randperm(x.size(0))] for key, x in x_dict.items()},
        edge_index_dict
    )
)

In [14]:
## train module
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)
dgi_model = dgi_model.to(device)

optimizer = torch.optim.Adam(dgi_model.parameters(), lr=0.0002)

def train_dgi(model, data, optimizer, epochs=200):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        pos_z, neg_z, summary = model(data.x_dict, data.edge_index_dict)
        loss = model.loss(pos_z, neg_z, summary)
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch:02d} | Loss: {loss.item():.4f}")

train_dgi(dgi_model, data, optimizer, epochs=200)


Epoch 00 | Loss: 1.3938
Epoch 01 | Loss: 1.3898
Epoch 02 | Loss: 1.3866
Epoch 03 | Loss: 1.3835
Epoch 04 | Loss: 1.3798
Epoch 05 | Loss: 1.3787
Epoch 06 | Loss: 1.3752
Epoch 07 | Loss: 1.3718
Epoch 08 | Loss: 1.3690
Epoch 09 | Loss: 1.3668
Epoch 10 | Loss: 1.3643
Epoch 11 | Loss: 1.3621
Epoch 12 | Loss: 1.3593
Epoch 13 | Loss: 1.3568
Epoch 14 | Loss: 1.3539
Epoch 15 | Loss: 1.3511
Epoch 16 | Loss: 1.3500
Epoch 17 | Loss: 1.3443
Epoch 18 | Loss: 1.3405
Epoch 19 | Loss: 1.3409
Epoch 20 | Loss: 1.3344
Epoch 21 | Loss: 1.3291
Epoch 22 | Loss: 1.3257
Epoch 23 | Loss: 1.3225
Epoch 24 | Loss: 1.3191
Epoch 25 | Loss: 1.3135
Epoch 26 | Loss: 1.3080
Epoch 27 | Loss: 1.3090
Epoch 28 | Loss: 1.3003
Epoch 29 | Loss: 1.2996
Epoch 30 | Loss: 1.2942
Epoch 31 | Loss: 1.2856
Epoch 32 | Loss: 1.2822
Epoch 33 | Loss: 1.2788
Epoch 34 | Loss: 1.2749
Epoch 35 | Loss: 1.2607
Epoch 36 | Loss: 1.2603
Epoch 37 | Loss: 1.2508
Epoch 38 | Loss: 1.2463
Epoch 39 | Loss: 1.2340
Epoch 40 | Loss: 1.2314
Epoch 41 | Loss:

In [15]:
file_prefix = "PPI_TOM_Module_eQTL/dgi_model_HAN_2layers_hierarchy_lr2e4_200epoch"

In [16]:
# Save model state dict
#torch.save(dgi_model.state_dict(), "../analysis/assets/module_assignment_network/"+file_prefix+".pth")
print("Model saved successfully!")

with torch.no_grad():
    embeddings = dgi_model.encoder(data.x_dict, data.edge_index_dict)

# Check the structure of embeddings
if isinstance(embeddings, dict):
    print("Embeddings keys:", embeddings.keys())  # Should include "gene"
    
    if "gene" in embeddings:
        print("Embeddings['gene'] shape:", embeddings["gene"].shape)
        gene_embeddings = embeddings["gene"].cpu().numpy()
    else:
        raise ValueError("Key 'gene' not found in embeddings.")
        
else:  # If embeddings is a single tensor, no need to index with 'gene'
    print("Embeddings is not a dict, using entire tensor.")
    gene_embeddings = embeddings.cpu().numpy()

print("Gene Embeddings Shape:", gene_embeddings.shape)  # Should be (num_nodes, hidden_dim)

import networkx as nx
from torch_geometric.utils import to_networkx
G = nx.Graph()

node_names = {
    "gene": data["gene"].names#,  # Assuming node names are stored
    #"go": data["go"].names
}
# Convert torch_geometric graph to NetworkX

# Add nodes to NetworkX with labels
for node_type, names in node_names.items():
    for i, name in enumerate(names):
        G.add_node(f"{node_type}_{i}", type=node_type, name=name)

# Convert edges
for edge_type, edge_index in data.edge_index_dict.items():
    src_type, relation, dst_type = edge_type
    edge_index = edge_index.cpu().numpy()
    # Weight 추가 (TOM/PPI/GO 관계마다 다를 수 있음)
    edge_weights = data[edge_type].edge_attr.cpu().numpy().flatten()

    for (src, dst, weight) in zip(edge_index[0], edge_index[1], edge_weights):
        G.add_edge(f"{src_type}_{src}", f"{dst_type}_{dst}", relation=relation, weight=weight)

# Save as edge list (CSV format)
#edge_list = nx.to_pandas_edgelist(G)
#edge_list.to_csv("../analysis/assets/module_assignment_network/PPI_TOM_eQTL/dgi_model_HAN_2layers_lr2e4_200epoch.csv", index=False)

# Save as GraphML or GML for direct import into R
#nx.write_graphml(G, "../analysis/assets/module_assignment_network/"+file_prefix+".graphml")

df_gene_embeddings = pd.DataFrame(gene_embeddings)
df_gene_embeddings.index = data["gene"].names
#df_gene_embeddings.to_csv("../analysis/assets/module_assignment_network/"+file_prefix+".tsv.gz", sep = "\t")


Model saved successfully!
Embeddings is not a dict, using entire tensor.
Gene Embeddings Shape: (2615, 64)


In [47]:
module_df.to_csv("../analysis/assets/module_assignment_network/"+file_prefix+"_moduleembedding.tsv.gz", sep = "\t")

In [38]:
from networkx.algorithms.centrality import eigenvector_centrality
import networkx as nx

G = nx.Graph()
for edge in ppi_edge_index.T.tolist():
    G.add_edge(common_genes[edge[0]], common_genes[edge[1]])

ec = eigenvector_centrality(G)
ec_values = np.array([ec[gene] if gene in ec else 0 for gene in data["gene"].names])


# 비교
import scipy.stats
correlation, pval = scipy.stats.spearmanr(ec_values, np.linalg.norm(gene_embeddings, axis=1))
print(f"Spearman correlation: {correlation:.4f}, p-value: {pval:.4e}")


Spearman correlation: 0.2792, p-value: 4.8748e-48


In [41]:
module_genes = module_df[module_df["module"] == "brown"]["gene_name"].values
module_embeddings = gene_embeddings[np.isin(data["gene"].names, module_genes)]

In [44]:
pd.DataFrame(module_embeddings, index = module_genes)

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
ABHD17A,-0.300179,0.137569,0.261652,-0.075853,0.166381,0.131724,0.254669,-0.183976,0.336819,-0.149537,...,-0.145318,-0.206203,0.199884,-0.097787,0.015048,-0.246346,-0.191160,-0.007187,0.154420,-0.026405
ABI3,-0.441602,0.274306,0.346450,-0.246595,0.314622,0.383928,0.434003,-0.285416,0.637197,-0.295935,...,-0.255691,-0.382908,0.423442,-0.301198,-0.141167,-0.457048,-0.323813,-0.041822,0.222234,-0.147580
ACAA2,-0.264870,0.118533,0.240402,-0.062944,0.156140,0.126572,0.253285,-0.180750,0.305656,-0.159245,...,-0.140336,-0.203008,0.199060,-0.097091,0.019084,-0.225547,-0.185464,0.010413,0.156449,-0.041054
ACTB,-0.294920,0.142536,0.251707,-0.094206,0.195031,0.187429,0.299303,-0.188728,0.357577,-0.209110,...,-0.174392,-0.252127,0.257003,-0.146155,-0.011180,-0.273348,-0.219319,0.022049,0.173095,-0.053520
ACTN4,-0.295488,0.173578,0.259273,-0.086216,0.211026,0.231656,0.339648,-0.199336,0.384944,-0.241595,...,-0.196666,-0.298019,0.271843,-0.169481,-0.024549,-0.277458,-0.220702,0.010418,0.198573,-0.096684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZDHHC24,-0.203623,0.065864,0.191002,-0.002175,0.096693,0.072215,0.212821,-0.153083,0.227330,-0.146388,...,-0.108034,-0.182625,0.123329,-0.033790,0.035153,-0.153209,-0.175157,0.036870,0.148863,-0.017272
ZEB2,-0.267998,0.125473,0.234346,-0.076397,0.146242,0.119711,0.264548,-0.186966,0.291843,-0.168485,...,-0.153481,-0.195600,0.230783,-0.117108,0.011910,-0.241102,-0.192241,0.051686,0.148377,0.002529
ZNF428,-0.202765,0.074519,0.196145,0.003616,0.101539,0.083075,0.227929,-0.150659,0.235697,-0.156353,...,-0.111158,-0.197385,0.125020,-0.041884,0.028598,-0.155256,-0.178723,0.038162,0.154811,-0.025718
ZNF683,-0.200033,0.067443,0.190725,0.005025,0.092179,0.075509,0.217757,-0.149135,0.228258,-0.149131,...,-0.109148,-0.186305,0.120590,-0.036816,0.033762,-0.150016,-0.177631,0.042280,0.149690,-0.017637
